<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización

In [ ]:
import numpy as np

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

##DATOS

In [ ]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [ ]:
def palabras_en_lista(corpus):
    palabras = []
    for elemento in corpus:
        terminos = elemento.split()
        for palabra in terminos:
          # Lista de palabras
          palabras.append(palabra)
    # Nos quedamos con una lista de elementos únicos
    vocabulario = set(palabras)
    return vocabulario

In [ ]:

vocabulario = palabras_en_lista(corpus)
print('\nVOCABULARIO DEL CORPUS: ', vocabulario)


VOCABULARIO DEL CORPUS:  {'hoy', 'gracias', 'es', 'dia', 'martes', 'de', 'muchas', 'el', 'que'}


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [ ]:
# One Hot Encoding
def one_hot_encoding(vocabulario):
    matriz_one_hot = []
    # Creamos una lista con las palabras únicas extraidas del corpus, posteriormente para cada elemento del corpus
    # vamos a traducirlo en un vector de 1's y 0's
    palabras_unicas = list(vocabulario)
    # Para cada elemento (documento) del corpus:
    for doc in corpus:
        doc_terms = doc.split()
        doc_encoding = []
        for term in doc_terms:
            index = palabras_unicas.index(term)
            # Cuando en la función palabras_en_lista hacemos vocabulario = set(palabras), creamos una lista aleatoria
            # de elementos únicos. Bajo esa alaeatoriedad encodeamos.
            doc_encoding.append(np.eye(len(palabras_unicas))[index])
        matriz_one_hot.append(np.array(doc_encoding))
    return matriz_one_hot



In [ ]:
# Codificamos
matriz_one_hot = one_hot_encoding(vocabulario)

In [ ]:
# Print
i = 0
for doc in corpus:
  print('\nDocumento:', doc)
  print("Equivalnte one hot: \n", matriz_one_hot[i])
  i += 1


Documento: que dia es hoy
Equivalnte one hot: 
 [[0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]]

Documento: martes el dia de hoy es martes
Equivalnte one hot: 
 [[0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]]

Documento: martes muchas gracias
Equivalnte one hot: 
 [[0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [ ]:
def freq_vector(matriz_one_hot):
    freq = []
    for doc in matriz_one_hot:
        one_hot = np.sum(doc, axis=0).astype(int)
        freq.append(one_hot)
    return np.array(freq)

In [ ]:
# Frecuencia de palabras
freq = freq_vector(matriz_one_hot)
print(freq)

[[1 0 1 1 0 0 0 0 1]
 [1 0 1 1 2 1 0 1 0]
 [0 1 0 0 1 0 1 0 0]]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [ ]:
# TF-IDF del corpus
def tf_idf(matriz_one_hot):
    encoded = []
    for doc in matriz_one_hot:
        one_hot = np.any(doc, axis=0).astype(int)
        encoded.append(one_hot)
    IDF = np.log10(np.divide(len(matriz_one_hot), np.sum(encoded, axis=0).astype(int)))
    return freq_vector(matriz_one_hot) * IDF

In [ ]:
TF_IDF = tf_idf(matriz_one_hot)
print(TF_IDF)

[[0.17609126 0.         0.17609126 0.17609126 0.         0.
  0.         0.         0.47712125]
 [0.17609126 0.         0.17609126 0.17609126 0.35218252 0.47712125
  0.         0.47712125 0.        ]
 [0.         0.47712125 0.         0.         0.17609126 0.
  0.47712125 0.         0.        ]]


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [ ]:
# Similitud coseno
def compare_docs(corpus, idx):
    vocabulario = palabras_en_lista(corpus)
    matriz_one_hot = one_hot_encoding(vocabulario)
    TF_IDF = tf_idf(matriz_one_hot)
    doc = TF_IDF[idx]
    # Vector de similitudes
    similar = []

    for i, doc_i in enumerate(TF_IDF):
        if i != idx:
            similarity =  cosine_similarity(doc, doc_i)
            similar.append((i, similarity))
    # Ordenamos
    similar.sort(key=lambda x: x[1], reverse=True)


    # Print
    for doc_idx, similarity in similar:
        print(f"Documento {idx} ({corpus[idx]}) con {doc_idx} ({corpus[doc_idx]}) = {similarity:.4f}")

    ordered_docs = [corpus[i] for i, _ in similar]
    print(f"Documentos ordenados por similitud coseno con el documento {idx}: {ordered_docs}\n")
    return ordered_docs

In [ ]:
print('\nSimilitud coseno por documento: \n')
for doc in range(3):
    ordered_docs = compare_docs(corpus, doc)


Similitud coseno por documento: 

Documento 0 (que dia es hoy) con 1 (martes el dia de hoy es martes) = 0.2003
Documento 0 (que dia es hoy) con 2 (martes muchas gracias) = 0.0000
Documentos ordenados por similitud coseno con el documento 0: ['martes el dia de hoy es martes', 'martes muchas gracias']

Documento 1 (martes el dia de hoy es martes) con 0 (que dia es hoy) = 0.2003
Documento 1 (martes el dia de hoy es martes) con 2 (martes muchas gracias) = 0.1085
Documentos ordenados por similitud coseno con el documento 1: ['que dia es hoy', 'martes muchas gracias']

Documento 2 (martes muchas gracias) con 1 (martes el dia de hoy es martes) = 0.1085
Documento 2 (martes muchas gracias) con 0 (que dia es hoy) = 0.0000
Documentos ordenados por similitud coseno con el documento 2: ['martes el dia de hoy es martes', 'que dia es hoy']

